<img style="float: left; margin:0px 15px 15px 0px; width:120px" src="https://www.orfeo-toolbox.org/wp-content/uploads/2016/03/logo-orfeo-toolbox.png">

# OTB Guided Tour - Virtual Workshop
## Emmanuelle SARRAZIN, Yannick TANGUY and David YOUSSEFI (CNES, French Space Agency)

<br>

Orfeo ToolBox (OTB) is an open-source library for remote sensing images processing. It has been initiated and funded by CNES to promote the use and the exploitation of the satellites images. Orfeo ToolBox aims at enabling large images state-of-the-art processing even on limited resources laptops, and is shipped with a set of extensible ready-to-use tools for classical remote sensing tasks, as well as a fully integrated, end-users oriented software called Monteverdi ; OTB is also accessible via QGIS processing module.

This tutorial will present the ORFEO Toolbox and showcase available applications for processing and manipulating satellite imagery.

<b> Press <span style="color:black;background:yellow">SHIFT+ENTER</span> to execute the notebook interactively cell by cell </b></div>



## Mount Google Drive and OTB Installation (Execution from Google Colaboratory)

The following cells are needed to run this notebook on Google Colab. First it mounts a google drive folder, then it downloads OTB binaries and install it in the virtual environment. Then, it compiles Python bindings so we can later run a "import otpApplication" command.

**Mount your google drive (follow the link, allow what is needed and get the authorization code)**

*if you run this notebook from your own computer, you can jump to cell "Python imports and definition of some support functions"*

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
# *******************************************************************************************************
# 
# At this step, google will ask you to login and authorize access to your google drive from this notebook
#
# *******************************************************************************************************

import sys

# This will download Orfeo ToolBox
!wget https://www.orfeo-toolbox.org/packages/archives/OTB/OTB-7.0.0-Linux64.run
!apt-get install file

# This configures OTB (source environment and compile Python bindings)
!chmod +x OTB-7.0.0-Linux64.run && ./OTB-7.0.0-Linux64.run && cd OTB-7.0.0-Linux64 && ctest -S share/otb/swig/build_wrapping.cmake -VV

In [ ]:
# *******************************************************************************************************
# Configure OTB environment variables
# *******************************************************************************************************
import os, sys
os.environ["CMAKE_PREFIX_PATH"] = "/content/OTB-7.0.0-Linux64"
os.environ["OTB_APPLICATION_PATH"] = "/content/OTB-7.0.0-Linux64/lib/otb/applications"
os.environ["PATH"] = "/content/OTB-7.0.0-Linux64/bin" + os.pathsep + os.environ["PATH"]
sys.path.insert(0, "/content/OTB-7.0.0-Linux64/lib/python")
os.environ["LC_NUMERIC"] = "C"
os.environ["GDAL_DATA"] = "/content/OTB-7.0.0-Linux64/share/gdal"
os.environ["PROJ_LIB"] = "/content/OTB-7.0.0-Linux64/share/proj"
os.environ["GDAL_DRIVER_PATH"] = "disable"
os.environ["OTB_MAX_RAM_HINT"] = "1000"

In [ ]:
# First upgrade pip
!pip install --upgrade pip folium
# Installation of third-parties libraries
!pip install rasterio==1.1.8 geojson

## Python imports and definition of some support functions

In [ ]:
#@title
# Imports
import datetime
import folium
import geojson
import io 
import os
import rasterio
import requests
import zipfile
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from rasterio import features
from rasterio import warp
from rasterio.warp import transform_bounds
from rasterio.warp import transform
from shapely.geometry import Polygon

# ignore rasterio FurtureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
#@title
"""
Some functions for Jupyter displaying
- quicklook creation
- map displaying (imshow, folium)
"""
def normalize(array, quantile=2):
    """
    normalizes bands into 0-255 scale
    :param array: numpy array to normalize
    :param quantile: quantile for ignoring outliers
    """
    array = np.nan_to_num(array)
    array_min, array_max = np.percentile(array, quantile), np.percentile(array, 100-quantile)
    normalized = 255*((array - array_min)/(array_max - array_min))
    normalized[normalized>255] = 255
    normalized[normalized<0] = 0
    return normalized.astype(np.uint8)


def imshow_RGBPIR(raster, colors=["Red", "Blue", "Green", "NIR"]):
    """
    show R,G,B, PIR image
    :param array: raster (rasterio image)
    :param colors: colors names list
    """
    print ("Quicklook by channel")
    multiband = []
    fig, ax = plt.subplots(1,len(colors)+1, figsize=(21,7))
    for nfig, color in enumerate(colors):
        band = raster.read(nfig+1, out_shape=(int(raster.height/16), int(raster.width/16)))
        band = normalize(band)

        if nfig < 3: multiband.append(band)

        try:          
            ax[nfig].imshow(band, cmap=color+'s')
        except ValueError:
            ax[nfig].imshow(band, cmap='Oranges')
    
        ax[nfig].set_title(color+' channel')

    ax[len(colors)].imshow(np.dstack(multiband))
    ax[len(colors)].set_title(colors[0]+colors[1]+colors[2])


def write_quicklook(raster, filename, downfactor=4):
    """
    write a JPG preview
    :param raster: raster (rasterio image)
    :param filename: path to the output preview
    :param downfactor: downsampling factor
    """
    profile = raster.profile

    # update size in profile
    newwidth = int(raster.width/downfactor)
    newheight = int(raster.height/downfactor)

    try:
        aff = raster.affine
        newaffine = rasterio.Affine(aff.a/downfactor, aff.b, aff.c,
                                    aff.d, aff.e/downfactor, aff.f)

        profile.update(dtype=rasterio.uint8, count=3, compress='lzw', driver='JPEG',
                       width=newwidth, height=newheight, transform=newaffine, affine=newaffine)

    # depend on rasterio version
    except AttributeError:
        aff = raster.transform
        newaffine = rasterio.Affine(aff.a/downfactor, aff.b, aff.c,
                                    aff.d, aff.e/downfactor, aff.f)

        profile.update(dtype=rasterio.uint8, count=3, compress='lzw', driver='JPEG',
                       width=newwidth, height=newheight, transform=newaffine)

    # write raster
    with rasterio.open(filename, 'w', **profile) as dst:
        for n in range(3):
            if raster.count == 1:
                band = raster.read(1, out_shape=(int(raster.height/downfactor), int(raster.width/downfactor)))
            else:
                band = raster.read(n+1, out_shape=(int(raster.height/downfactor), int(raster.width/downfactor)))
                
            band = normalize(band)
            dst.write(band, n+1)


def rasters_on_map(rasters_list, out_dir, overlay_names_list=None, geojson_data=None):
    """
    displays a raster on a ipyleaflet map
    :param rasters_list: rasters to display (rasterio image)
    :param out_dir: path to the output directory (preview writing)
    :param overlay_names_list: name of the overlays for the map
    """
    # - Fill overlay names list
    if overlay_names_list is None:
      overlay_names_list = [ "Image "+str(i) for i in range(len(rasters_list))]
    
    # - get bounding box
    raster = rasters_list[0]
    epsg4326 = {'init': 'EPSG:4326'}
    bounds = transform_bounds(raster.crs, epsg4326, *raster.bounds)
    center = [(bounds[0]+bounds[2])/2, (bounds[1]+bounds[3])/2]

    # - get centered map
    f = folium.Figure(width=1100, height=600)
    m = folium.Map(location=(center[-1], center[0]),start_zoom=10).add_to(f)
    
    # - plot quicklook
    for raster, overlay_name in zip(rasters_list, overlay_names_list):
        bounds = transform_bounds(raster.crs, epsg4326, *raster.bounds)
        quicklook_url = os.path.join(out_dir, "PREVIEW_{}.JPG".format(datetime.datetime.now()))
        write_quicklook(raster, quicklook_url)
        quicklook = folium.raster_layers.ImageOverlay(
            quicklook_url,
            ((bounds[1], bounds[0]),(bounds[3], bounds[2])),
            name=overlay_name
        )
        m.add_children(quicklook)

    # - add geojson data
    if geojson_data is not None:
        folium.GeoJson(
                       data=geojson_data,
                       name="json",
                       #style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1},
                      ).add_to(m)

    # - add layer control
    folium.LayerControl().add_to(m)
    
    return f

def get_bounding_box_from_draw(raster, dc):
    """
    returns bounding box in a image
    :param raster: displayed raster (rasterio image)
    :param dc: drawcontrol ipyleaflet
    """
    try:
        # Get last draw from the map
        coordinates = np.array(dc.last_draw['geometry']['coordinates'][0])

        # lon, lat to Sentinel-2 coordinate reference system
        lon_min_max = [np.amin(coordinates[:,0]), np.amax(coordinates[:,0])]
        lat_min_max = [np.amin(coordinates[:,1]), np.amax(coordinates[:,1])]
        lons, lats = np.meshgrid(lon_min_max, lat_min_max)
        xs, ys = transform({'init': 'EPSG:4326'}, raster.crs, lons.flatten(), lats.flatten())

        # Get the region of interest in the image
        rows, cols = [], []
        for x, y in zip(xs, ys):
            row, col = ~raster.affine * (x, y)
            rows.append(row)
            cols.append(col)
        row_min, row_max = np.amin(rows), np.amax(rows)
        col_min, col_max = np.amin(cols), np.amax(cols)
        startx, starty = map(lambda v:int(np.floor(v)), [col_min, row_min])
        endx, endy = map(lambda v:int(np.ceil(v)), [col_max, row_max])
        print ("Bounding box computed")
    except:
        print ("Draw a polygon in the displayed map")
        startx, starty, endx, endy = None, None, None, None
    return startx, starty, endx, endy

## Configure access to the Sentinel 2 Dataset

### Configure dataset location and output directory


In [ ]:
# Execution from COLAB -> use a folder on google drive
OTB_ROOT = "gdrive/MyDrive/OTB_formation/01-guided-tour"
Path(OTB_ROOT).mkdir(parents=True, exist_ok=True)

# Data directory
DATA_DIR = os.path.join(OTB_ROOT,"data")

# Output directory
OUTPUT_DIR = os.path.join(OTB_ROOT,"output")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# Local execution
# /!\ Please enter the right location !! 
#DATA_DIR = "/home/your_login/OTB"
# Local execution
#OUTPUT_DIR = "/home/your_login/your_output_folder"

In [ ]:
zip_file_url = "https://www.orfeo-toolbox.org/packages/WorkshopData/data_otb-guided-tour.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(OTB_ROOT)

In [ ]:
# Check if data has been correctly downloaded
!ls gdrive/MyDrive/OTB_formation/data

## View images

In [ ]:
image1 = os.path.join(DATA_DIR,"xt_SENTINEL2B_20180621-111349-432_L2A_T30TWT_D_V1-8_RVBPIR.tif")
image2 = os.path.join(DATA_DIR,"xt_SENTINEL2B_20180701-111103-470_L2A_T30TWT_D_V1-8_RVBPIR.tif")
image3 = os.path.join(DATA_DIR,"xt_SENTINEL2B_20180711-111139-550_L2A_T30TWT_D_V1-8_RVBPIR.tif")

In [ ]:
im = rasterio.open(image1)
rasters_on_map(
               [im],  # Image list
               OUTPUT_DIR, # Directory to save quicklooks
               ["Image 1"]  # Image labels for legend
               )

## 1) How to compute vegetation index with OTB

Here we create an application with otbApplication.Registry.CreateApplication("BandMath")

[BandMath](https://www.orfeo-toolbox.org/CookBook/Applications/app_BandMath.html) takes a list of images as input, so we have to give a Python list with "il" parameter : [image], or [image1, image2, .., imageN] and the main parameter is the mathematical expression "exp".

We will compute the Normalized Difference Vegetation Index [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index).

The formula for NDVI is:

$NDVI=\frac{X_{nir}-X{red}}{X_{nir}+X_{red}}$  

where
* $X_{nir}$ is the NIR (Near-Infrared) band
* $X_{red}$ is the red band

For Sentinel2 data, the corresponding bands for NIR and Red are respectively the 4th and the 1st bands (b4, b1) of the image.

In [ ]:
import otbApplication as otb

In [ ]:
def compute_ndvi(im, ndvi):
    """
    Compute NDVI for a image and write the result

    :param im: Path to input image
    :param ndvi: Path to ndvi image
    """
    app = otb.Registry.CreateApplication("BandMath")
    app.SetParameterStringList("il",[im])
    app.SetParameterString("out", ndvi)
    app.SetParameterString("exp", "(im1b4-im1b1)/(im1b4+im1b1)")
    exit_code = app.ExecuteAndWriteOutput()

In [ ]:
ndvi1 = os.path.join(DATA_DIR,"ndvi1.tif")
ndvi2 = os.path.join(DATA_DIR,"ndvi2.tif")
ndvi3 = os.path.join(DATA_DIR,"ndvi3.tif")

In [ ]:
# TODO: Choose an image an compute the NDVI ()
compute_ndvi(...,....)


In [ ]:
#@title
compute_ndvi(image1,ndvi1)

In [ ]:
# display the result
im = rasterio.open(image1)
ndvi = rasterio.open(ndvi1)
rasters_on_map([im, ndvi],OUTPUT_DIR,["Image", "NDVI"])

## 2) Compute Normalized Difference Water Index from Green and Near InfraRed bands

We will compute the Normalized Difference Water Index [NDWI](https://en.wikipedia.org/wiki/Normalized_difference_water_index).
We will use the formula defined by McFeeters in 1996 which is obtained from green and nir bands :

$NDWI2=\frac{X_{green}−X_{nir}}{X_{green}+X_{nir}}$  

where
* $X_{nir}$ is the NIR (Near-Infrared) band
* $X_{green}$ is the green band

For Sentinel2 data, the corresponding bands for NIR and Green are respectively the 4th and the 2nd bands (b4, b2) of the image.

In [ ]:
# TODO: Complete the expression for NDWI
def compute_ndwi(im, ndwi):
    """
    Compute NDWI for a image and write the result

    :param im: Path to input image
    :param ndwi: Path to ndwi image
    """
    app = otb.Registry.CreateApplication("BandMath")
    app.SetParameterStringList("il",[im])
    app.SetParameterString("out", ndwi)
    app.SetParameterString("exp", "........")
    exit_code = app.ExecuteAndWriteOutput()

In [ ]:
#@title
def compute_ndwi(im, ndwi):
    """
    Compute NDWI for a image and write the result

    :param im: Path to input image
    :param ndwi: Path to ndwi image
    """
    app = otb.Registry.CreateApplication("BandMath")
    app.SetParameterStringList("il",[im])
    app.SetParameterString("out", ndwi)
    app.SetParameterString("exp", "(im1b2-im1b4)/(im1b2+im1b4)")
    exit_code = app.ExecuteAndWriteOutput()

In [ ]:
# Compute NDWI images
ndwi1 = os.path.join(DATA_DIR,"ndwi1.tif")
ndwi2 = os.path.join(DATA_DIR,"ndwi2.tif")
ndwi3 = os.path.join(DATA_DIR,"ndwi3.tif")

compute_ndwi(image1,ndwi1)
compute_ndwi(image2,ndwi2)
compute_ndwi(image3,ndwi3)

In [ ]:
# Display NDWI images 
rasters_on_map(
    [rasterio.open(ndwi1),rasterio.open(ndwi2),rasterio.open(ndwi3)],
    OUTPUT_DIR,
    ["NDWI 1","NDWI 2","NDWI 3"]
    )

## 3) Compute water mask

The aim of this exercise is to use NDWI2 values to create a water mask.

For NDWI2, a threshold can be found in https://www.mdpi.com/2072-4292/5/7/3544/htm (McFeeters, 2013):

* < 0.3   : Non-water
* \>= 0.3 : Water

*We will have to check if this threshold has to be adapted or not !*



### Create a simple watermask (NDWI threshold)

The objective is to compute a watermask with one image using a threshold.

OTB BandMath can use these formula :

    binary operators:
        ‘+’ addition, ‘-‘ subtraction, ‘*’ multiplication, ‘/’ division
        ‘^’ raise x to the power of y
        ‘<’ less than, ‘>’ greater than, ‘<=’ less or equal, ‘>=’ greater or equal
        ‘==’ equal, ‘!=’ not equal
        ‘||’ logical or, ‘&&’ logical and
    functions: exp(), log(), sin(), cos(), min(), max(), ...
    if-then-else : "(<expression> ? <value if true> : <value if false>)"

https://www.orfeo-toolbox.org/CookBook/Applications/app_BandMath.html

Try to write an expression to create a basic watermask :

*if ndwi < 0.1 then return 0 else return 1*

And then, try another threshold value

In [ ]:
# TODO: Complete the method
def threshold_ndwi(ndwi, mask):
    """
    Compute mask with threshold for a NDWI image and write the mask result
    
    :param ndwi: Path to ndwi image
    :param mask: Path to mask image
    """
    # TODO: Fill the threshold_ndwi function
    pass

In [ ]:
mask1 = os.path.join(OUTPUT_DIR,"mask1.tif")
threshold_ndwi(ndwi1, mask1)

In [ ]:
#@title
def threshold_ndwi(ndwi, mask):
    """
    Compute mask with threshold for a NDWI image and write the mask result
    
    :param ndwi: Path to ndwi image
    :param mask: Path to mask image
    """
    app = otb.Registry.CreateApplication("BandMath")
    app.SetParameterStringList("il",[ndwi])
    app.SetParameterString("out", mask)
    app.SetParameterString("exp", "(im1b1 < 0.1 ? 0 : 1) ")
    exit_code = app.ExecuteAndWriteOutput()

In [ ]:
# Compute masks
mask1 = os.path.join(DATA_DIR,"mask1.tif")
mask2 = os.path.join(DATA_DIR,"mask2.tif")
mask3 = os.path.join(DATA_DIR,"mask3.tif")

threshold_ndwi(ndwi1,mask1)
threshold_ndwi(ndwi2,mask2)
threshold_ndwi(ndwi3,mask3)

In [ ]:
# Display masks
rasters_on_map(
    [rasterio.open(mask1),rasterio.open(mask2),rasterio.open(mask3)],
    OUTPUT_DIR,
    ["Mask 1","Mask 2","Mask 3"]
    )

### Create a watermask with the different NDWI images

As we have seen, the NDWI2 images are different depending on the dates, mainly because tide level is different and there maybe some clouds that hide some regions of the image.

We have to find a function that can combine the information from the different NDWI images to improve the watermask.

So, we now want to use the three dates to obtain a better watermask, that will better identify the water presence. To do so, we shall identify the largest areas (high tides) in the different watermasks.

*Tips : OTB BandMath can take as input a list of images (im1, im2, ...) and produce a single result*

In [ ]:
# TODO: Complete the method
def create_water_mask(ndwi1, ndwi2, ndwi3, mask):
    """
    Compute water mask and write the result
    
    :param ndwi1: Path to ndwi image 1
    :param ndwi2: Path to ndwi image 2
    :param ndwi3: Path to ndwi image 3
    :param mask: Path to mask image
    """
    pass

In [ ]:
#@title
def create_water_mask(ndwi1, ndwi2, ndwi3, mask):
    """
    Compute water mask and write the result
    
    :param ndwi1: Path to ndwi image 1
    :param ndwi2: Path to ndwi image 2
    :param ndwi3: Path to ndwi image 3
    :param mask: Path to mask image
    """
    app = otb.Registry.CreateApplication("BandMath")
    app.SetParameterStringList("il",[ndwi1, ndwi2, ndwi3])
    app.SetParameterString("out", mask)
    app.SetParameterString("exp", "(max(im1b1, im2b1, im3b1) < 0.1 ? 0 : 1) ")
    exit_code = app.ExecuteAndWriteOutput()

In [ ]:
# Compute mask
final_mask = os.path.join(OUTPUT_DIR,"final_mask.tif")
create_water_mask(ndwi1, ndwi2, ndwi3, final_mask)

In [ ]:
rasters_on_map(
    [rasterio.open(image1),rasterio.open(image2),rasterio.open(image3),rasterio.open(final_mask)],
    OUTPUT_DIR,
    ["Image 1","Image 2","Image 3","Water mask"]
    )

## 4) Polygonize watermask and filter features to count islands

In this step, we are going to polygonize our binary masks : we will obtain a lot of polygons ! Some of these features have to be filtered (main land, ocean) in order to count the islands in Morbihan gulf.

In [ ]:
# Morbihan gulf
morbihan = {'type': 'FeatureCollection', 
            'features': [{'type': 'Feature', 
                          'properties': {}, 
                          'geometry': {'type': 'Polygon', 
                                       'coordinates': [[[-2.953968, 47.603544], 
                                                        [-2.958085, 47.589653], 
                                                        [-2.929956, 47.563713], 
                                                        [-2.883303, 47.561397], 
                                                        [-2.86478, 47.556763], 
                                                        [-2.840081, 47.547958], 
                                                        [-2.826638, 47.552744], 
                                                        [-2.808114, 47.55761], 
                                                        [-2.774497, 47.5495], 
                                                        [-2.749113, 47.546951], 
                                                        [-2.730932, 47.564329], 
                                                        [-2.728188, 47.5861], 
                                                        [-2.740537, 47.595825], 
                                                        [-2.747055, 47.605317], 
                                                        [-2.780672, 47.609252], 
                                                        [-2.786846, 47.613649], 
                                                        [-2.802348, 47.61882], 
                                                        [-2.831848, 47.616969], 
                                                        [-2.857233, 47.620209], 
                                                        [-2.862721, 47.609563], 
                                                        [-2.865466, 47.600303], 
                                                        [-2.880559, 47.59984], 
                                                        [-2.891536, 47.597988], 
                                                        [-2.896339, 47.582243], 
                                                        [-2.938875, 47.589653], 
                                                        [-2.953968, 47.603544]]]
                                       }
                          }
                         ]
            }
morbihan_as_polygon = Polygon(morbihan['features'][0]['geometry']['coordinates'][0])
rasters_on_map([rasterio.open(image2)], OUTPUT_DIR, ["Image Sentinel 2"], geojson_data=morbihan)

In [ ]:
# Convert watermask to geojson collection
with rasterio.open(final_mask) as src:
    image = src.read(1).astype(np.uint8)
try:
    transform = src.affine
# depend on rasterio version
except AttributeError:
    transform = src.transform
results = list({'type':'Feature', 
            'properties': {}, 
            'geometry': s} 
           for i, (s, __) in enumerate(features.shapes(image, mask=image, transform=transform)))     

# Visualize
collection = {'type': 'FeatureCollection', 'features': list()}
for res in results:
    feature = dict(res)
    # convert geom to EPSG:4326 (WGS84)
    feature['geometry'] = warp.transform_geom(src.crs, 'EPSG:4326', res['geometry'])
    collection['features'].append(feature)
rasters_on_map([rasterio.open(image2)], OUTPUT_DIR, ["Image Sentinel 2"], geojson_data=collection) 

In [ ]:
# Filter geojson
filtered_collection = {'type': 'FeatureCollection', 'features': list()}
areas = []
polygon = None
for res in results:
    # convert geom to EPSG:4326 (WGS84)
    geom_for_geojson = warp.transform_geom(src.crs, 'EPSG:4326', res['geometry'])
    
    # Loop for multiple polygons
    for i in range(len(res['geometry']['coordinates'])):
        # area in m^2
        item_area = Polygon(res['geometry']['coordinates'][i]).area          
        island_as_polygon = Polygon(geom_for_geojson['coordinates'][i])
    
        # Filter the smallest areas and the biggest (main land) and
        # crop the "watermask" with envelope shape morbihan (~ Morbihan gulf)   
        if item_area < 5000000.0 and item_area > 7000.0 and island_as_polygon.intersects(morbihan_as_polygon):
            polygon = island_as_polygon
            feature = {'type': 'Feature',
                       'properties':{},
                       'geometry':{'type':'Polygon',
                                   'coordinates': [geom_for_geojson['coordinates'][i]]
                                   }
            }
            filtered_collection['features'].append(feature)
            areas.append(item_area)

In [ ]:
print ("Nb islands: {}".format(len(filtered_collection['features'])))

### Visualize the islands (and count them :-)

In [ ]:
rasters_on_map([rasterio.open(image2)], OUTPUT_DIR, ["Image Sentinel 2"], geojson_data=filtered_collection)

## Extra steps

We could optimize the previous code by using OTB pipeline : instead of computing 3 watermaks, and then combining them, we could simplify the processing chain and compute directly the final watermask. This will save I/O and thus save computation time (especially if the chain is complex or use a lot of images).

Since OTB 5.8, it is possible to connect an output image parameter from one application to the input image parameter of the next parameter. This results in the wiring of the internal ITK/OTB pipelines together, permitting image streaming between the applications. Consequently, this removes the need of writing temporary images and improves performance. Only the last application of the processing chain is responsible for writing the final result images.

<b> Please rewrite the  <span style="color:black;background:yellow"> code bellow </span> in order to only write the watermask file </b>

**Tips:** Only call Execute() to setup the pipeline, not ExecuteAndWriteOutput() which would run it and write the output image and also use these functions to connect OTB applications :
- ```GetParameterOutputImage``` : get a pointer to an image object [instead of reading from file]
- ```AddImageToParameterInputImageList``` : add an image to an InputImageList parameter as an pointer to an image object pointer [instead of reading from file] (```SetParameterInputImage``` for an InputImageList)